In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
name = 'twitter15'

In [35]:
df_train = pd.read_csv(f'seiz_dataset_{name}.csv')
df_val = pd.read_csv(f"seiz_dataset_{name}_val.csv")

In [102]:
df_train.shape

(994, 14)

In [36]:
init_pops = ['S0', 'E0', 'I0', 'Z0']

In [37]:
def transform_seiz_data(input_data):
    #Computing R score
    df = input_data.copy()
    p = df.p.values
    beta = df.beta.values
    l = df.l.values
    rho = df.rho.values
    epsilon = df.epsilon.values
    b = df.b.values
    df['R'] = ((1 - p) * beta + (1 - l) * b) / (rho + epsilon)
    # Normalizing population columns
    df['N'] = df[init_pops].values.sum(1)
    for col in init_pops:
        df[col] = df[col].values / df['N'].values
    df = df.drop(columns='N')
    #Incubation time is relevant
    df['incubation_time'] = 1 / df.epsilon.values
    #Encode target as category for lightgbm
    encoder = {"true":0, "false":1, "non-rumor":2, "unverified":3}
    df.label = df.label.apply(lambda x: encoder[x])
    return df

In [38]:
df_train = transform_seiz_data(df_train)
df_val = transform_seiz_data(df_val)

### Experimenting with LighGbm

In [93]:
import lightgbm as lgb
from sklearn.metrics import f1_score, accuracy_score

In [94]:
gbm_model = GradientBoostingModel(ntrees=1000, lr=0.01, max_d=5, feature_fraq=1.0)

dataset = lgb.Dataset(df_train.iloc[:, 1:], label=df_train.label)
dataset.construct()
params = {
    'objective':'multiclass',
    'num_class':4,
    'bagging_freq':5,
    'feature_fraction':1.0,
    'boosting_type':'gbdt',
    'max_depth':5,
    'learning_rate':0.01,
    'n_estimators':1000,
    'verbosity':2
    }
gbm_model = lgb.train(params, train_set=dataset)

preds = gbm_model.predict(df_val.iloc[:, 1:], raw_score=False).argmax(1)

accuracy_score(df_val.iloc[:, 0].values, preds)

lgb.plot_importance(gbm_model)

### Trying out a MLP

In [271]:
import torch
import torch.nn as nn

In [ ]:
n_hidden_layers = 3
hidden_size = 64
batch_size = 64
lr = 0.0001
dropout_prob =0.2
n_epochs = 10000

device='cuda' if torch.cuda.is_available() else 'cpu'

model_seq = [nn.BatchNorm1d(num_features=13, affine=False)]
model_seq.append(nn.Linear(in_features=13, out_features=hidden_size))
for _ in range(n_hidden_layers):
    model_seq.append(nn.Dropout(dropout_prob))
    model_seq.append(nn.Linear(in_features=hidden_size, out_features=hidden_size))
model_seq.append(nn.Linear(in_features=hidden_size, out_features=4))
mlp = nn.Sequential(*model_seq)
mlp = mlp.to(device)

optim = torch.optim.Adam([param for name, param in mlp.named_parameters()], lr=lr)

train_loader = torch.utils.data.DataLoader(df_train.values, batch_size=batch_size, shuffle=True)
test_loader = train_loader

loss_func = nn.CrossEntropyLoss(reduction='mean')
for epoch in range(n_epochs):
    epoch_loss = 0.
    
    #TRAIN
    mlp.train()
    for dp in train_loader:
        y = dp[:, 0].to(device).long()
        x = dp[:, 1:].to(device).float()
        logits = mlp(x)

        
        loss = loss_func(logits, y)
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        epoch_loss += loss.item()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}: Mean Loss = {epoch_loss/len(train_loader):.3f}")
        
    
    #EVAL
    mlp.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for dp in test_loader:
            y = dp[:, 0].to(device).long()
            x = dp[:, 1:].to(device).float()
            logits = mlp(x)
            _, preds = logits.max(dim=1)
            correct += float(preds.eq(y).sum().item())
            total += y.size(0)
    if (epoch+1) % 10 == 0:
        print(f"Accuracy Epoch {epoch+1}: {correct/total:.3f}")